In [1]:
import cv2
import sys
import ctypes
import subprocess
import platform
import mediapipe as mp
import util

In [2]:
mpHands = mp.solutions.hands
hands = mpHands.Hands(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7,
    max_num_hands=1
)
mpDraw = mp.solutions.drawing_utils 

I0000 00:00:1730911576.013661   12040 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1730911576.018710   12252 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.2.6-arch1.1), renderer: AMD Radeon Graphics (radeonsi, rembrandt, LLVM 18.1.8, DRM 3.54, 6.6.59-1-lts)


In [3]:
def get_screen_resolution():
    """Mendapatkan resolusi layar untuk Windows dan Linux"""
    system = platform.system()
    
    try:
        if system == "Windows":
            # Untuk Windows menggunakan ctypes
            user32 = ctypes.windll.user32
            screen_width = user32.GetSystemMetrics(0)
            screen_height = user32.GetSystemMetrics(1)
            
        elif system == "Linux":
            # Untuk Linux menggunakan xrandr
            cmd = ['xrandr']
            cmd2 = ['grep', '*']
            
            p = subprocess.Popen(cmd, stdout=subprocess.PIPE)
            p2 = subprocess.Popen(cmd2, stdin=p.stdout, stdout=subprocess.PIPE)
            p.stdout.close()
            
            resolution_string = p2.communicate()[0].decode('utf-8').strip()
            
            # Parse resolusi dari output xrandr
            resolution = resolution_string.split()[0]
            width, height = resolution.split('x')
            screen_width = int(width)
            screen_height = int(height)
            
        else:
            # Default resolution untuk sistem operasi lain
            print(f"Sistem operasi {system} tidak didukung, menggunakan resolusi default")
            screen_width = 1920
            screen_height = 1080
            
    except Exception as e:
        print(f"Gagal mendapatkan resolusi layar: {str(e)}")
        print("Menggunakan resolusi default")
        screen_width = 1920
        screen_height = 1080
    
    print(f"Resolusi layar terdeteksi: {screen_width}x{screen_height}")
    
    return screen_width, screen_height

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [4]:

def initialize_webcam(width=1280, height=720):
    """Inisialisasi webcam dengan resolusi tertentu"""
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise Exception("Tidak dapat mengakses webcam")
    
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)
    
    return cap

W0000 00:00:1730911576.037019   12233 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [5]:
def create_centered_window(window_name, width, height):
    """Membuat window di tengah layar"""
    cv2.namedWindow(window_name, cv2.WINDOW_GUI_NORMAL | cv2.WINDOW_AUTOSIZE)
    
    # Dapatkan resolusi layar
    screen_width, screen_height = get_screen_resolution()
    
    # Hitung posisi untuk window di tengah
    x = (screen_width - width) // 2
    y = (screen_height - height) // 2
    
    # Set posisi window
    cv2.moveWindow(window_name, x, y)
    
    # Set ukuran window
    cv2.resizeWindow(window_name, width, height)

In [6]:
def process_frame(frame, width, height):
    """Memproses frame dengan menambahkan overlay dan informasi"""
    # Flip frame horizontal
    frame = cv2.flip(frame, 1)
    
    # Resize frame ke ukuran tetap
    frame = cv2.resize(frame, (width, height))
    
    # Buat overlay untuk informasi
    overlay = frame.copy()
    cv2.rectangle(overlay, (0, 0), (width, 60), (0, 0, 0), -1)
    cv2.addWeighted(overlay, 0.3, frame, 0.7, 0, frame)
    
    # Tambahkan text judul
    cv2.putText(frame, "Virtual Mouse Control", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
    # Tambahkan panduan kontrol
    guide_text = "Tekan 'q' untuk keluar"
    cv2.putText(frame, guide_text, (10, height-20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    return frame

W0000 00:00:1730911576.054194   12235 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [7]:
def detect_hand_landmarks(frame):
    """Mendeteksi hand landmarks dan menggambar di frame"""
    frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    processed = hands.process(frameRGB)
    
    landmark_list = []
    
    if processed.multi_hand_landmarks:
        hand_landmarks = processed.multi_hand_landmarks[0]
        mpDraw.draw_landmarks(frame, hand_landmarks, mpHands.HAND_CONNECTIONS)
        
        for lm in hand_landmarks.landmark:
            normalized_x = lm.x * frame.shape[1]
            normalized_y = lm.y * frame.shape[0]
            landmark_list.append((normalized_x, normalized_y))
    
    return frame, landmark_list

In [8]:
def main():
    """Fungsi utama program"""
    WINDOW_NAME = 'Virtual Mouse Control'
    WIDTH = 1280
    HEIGHT = 720
    
    try:
        # Inisialisasi webcam
        cap = initialize_webcam(WIDTH, HEIGHT)
        
        # Buat window di tengah
        create_centered_window(WINDOW_NAME, WIDTH, HEIGHT)
        
        while True:
            # Baca frame dari webcam
            ret, frame = cap.read()
            if not ret:
                print("Gagal membaca frame dari webcam")
                break
                
            # Proses frame
            processed_frame = process_frame(frame, WIDTH, HEIGHT)
            
            # Deteksi hand landmarks
            processed_frame, landmarks = detect_hand_landmarks(processed_frame)

            # Tampilkan frame
            cv2.imshow(WINDOW_NAME, processed_frame)
            
            # Cek input keyboard dan status window
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q') or cv2.getWindowProperty(WINDOW_NAME, cv2.WND_PROP_VISIBLE) < 1:
                print("Program dihentikan")
                break
                
    except Exception as e:
        print(f"Terjadi error: {str(e)}")
        sys.exit(1)
        
    finally:
        # Bersihkan resource
        if 'cap' in locals():
            cap.release()
        cv2.destroyAllWindows()

if __name__ == '__main__':
    main()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""


Resolusi layar terdeteksi: 1920x1200


W0000 00:00:1730911584.759237   12231 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Program dihentikan
